In [ ]:


!pip install /kaggle/input/audiomentations-v0290/resampy-0.4.2-py3-none-any.whl
!pip install  /kaggle/input/audiomentations-v0290/librosa-0.9.2-py3-none-any.whl


!export OMP_NUM_THREADS=N

!export OMP_SCHEDULE=STATIC
!export OMP_PROC_BIND=CLOSE
!export GOMP_CPU_AFFINITY="N-M"



In [ ]:

!pip install /kaggle/input/openvino-wheels/openvino-2022.3.0-9052-cp37-cp37m-manylinux_2_17_x86_64.whl --no-index --find-links /kaggle/input/openvino-wheels

In [ ]:
import openvino.runtime as ov

In [ ]:
from pathlib import Path
import random

import numpy as np
import pandas as pd
import time
import os
import matplotlib.pyplot as plt
# These transformations will be passed to our model class
import torch
import torch.nn.functional as F
import torch.nn as nn
import yaml
from tqdm.auto import tqdm
import glob
from torch.distributions import Beta
import librosa

In [ ]:
import os
from os.path import exists

WAV_SIZE=16000
STEP_SIZE=500
TIMES_REAL=4
TIMES_TRAIN=8
is_mixed_precision = True
TARGET_COLS = ['StartHesitation', 'Turn', 'Walking']

class GaitDataset(torch.utils.data.Dataset):

    def __init__(self, df, is_train=False,transforms=None):
        self.is_train = is_train
        self.data = df

    def __len__(self):
        if self.is_train:
            return len(self.data)*TIMES_TRAIN
        else:
            return len(self.data)
    
    
    def __getitem__(self, idx):
        g0=9.80665
        row = self.data.iloc[idx]
        data = pd.read_csv(row.filename)
        
        print(row.Id, data.shape)
        
        sig = data[[ 'AccV', 'AccML', 'AccAP']].values
        
        if row.type == 0:
            sigs = []
            for c in range(3):
                sigs.append(librosa.resample(sig[:,c],orig_sr=128,target_sr=100))
            wav = np.stack(sigs,axis=1)
        else:
            wav = sig*g0
        
        print('after resampling',wav.shape)
        wav = wav/40.
        act_len = len(wav)
        nchunk = len(wav)//WAV_SIZE
        rem_size = len(wav) - nchunk*WAV_SIZE
        arrs = []
        for chk in range(nchunk):
            arrs.append(wav[chk*WAV_SIZE:(chk+1)*WAV_SIZE])

        
        if rem_size > 0:
            last_arr = wav[-WAV_SIZE:]
            arrs.append(last_arr)
            
        wav = np.stack(arrs,axis=0)
        
        print('wav',wav.shape,rem_size)
        
        sample = {"wav": wav, "Id":row.Id, 'type': row.type, 'df_length':len(data), 
                  'act_len':act_len,'nchunk':nchunk,'rem_size':rem_size}

        return sample
        

def getDataLoader(params,val_x):
    
    val_dataset = GaitDataset(df=val_x, transforms=None)

    valDataLoader = torch.utils.data.DataLoader(
                        val_dataset,
                        batch_size=1,
                        num_workers=params['num_workers'],
                        shuffle=False,
                        pin_memory=False,
                    )
    
    return valDataLoader

In [ ]:
class Wave_Block(nn.Module):

    def __init__(self, in_channels, out_channels, dilation_rates, kernel_size):
        super(Wave_Block, self).__init__()
        self.num_rates = dilation_rates
        self.convs = nn.ModuleList()
        self.filter_convs = nn.ModuleList()
        self.gate_convs = nn.ModuleList()

        self.convs.append(nn.Conv1d(in_channels, out_channels, kernel_size=1))
        dilation_rates = [2 ** i for i in range(dilation_rates)]
        for dilation_rate in dilation_rates:
            self.filter_convs.append(
                nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate))
            self.gate_convs.append(
                nn.Conv1d(out_channels, out_channels, kernel_size=kernel_size, padding=int((dilation_rate*(kernel_size-1))/2), dilation=dilation_rate))
            self.convs.append(nn.Conv1d(out_channels, out_channels, kernel_size=1))

    def forward(self, x):
        x = self.convs[0](x)
        res = x
        for i in range(self.num_rates):
            x = torch.tanh(self.filter_convs[i](x)) * torch.sigmoid(self.gate_convs[i](x))
            x = self.convs[i + 1](x)
            res = res + x
        return res
# detail 
class Classifier(nn.Module):
    def __init__(self, inch=3, kernel_size=3):
        super().__init__()
        self.LSTM = nn.GRU(input_size=128, hidden_size=128, num_layers=4, 
                           batch_first=True, bidirectional=True)
        
        #self.wave_block1 = Wave_Block(inch, 16, 12, kernel_size)
        self.wave_block2 = Wave_Block(inch, 32, 8, kernel_size)
        self.wave_block3 = Wave_Block(32, 64, 4, kernel_size)
        self.wave_block4 = Wave_Block(64, 128, 1, kernel_size)
        self.fc1 = nn.Linear(256, 3)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        #x = self.wave_block1(x)
        x = self.wave_block2(x)
        x = self.wave_block3(x)

        x = self.wave_block4(x)
        x = x.permute(0, 2, 1)
        x, h = self.LSTM(x)
        x = self.fc1(x)
    
        
        return x

In [ ]:
def save_model(epoch,model,ckpt_path='./',name='',val_rmse=0):
    path = os.path.join(ckpt_path, '{}_{}.pth'.format(name, epoch))
    torch.save(model.state_dict(), path, _use_new_zipfile_serialization=False)
    
def load_model(model,ckpt_path):
    state = torch.load(ckpt_path,map_location=torch.device('cpu'))
    print(model.load_state_dict(state,strict=False))
    return model

In [ ]:


def validation_step(model, batch, batch_idx):
    # Load images and labels
    x = batch["wav"].float()
    if GPU:
        x= x.cuda(non_blocking=True)
    x = x[0]
    
    print('x',x.shape)
    # Forward pass & softmax
    
    flat_pred = np.zeros((batch['act_len'][0],3))
    with torch.no_grad():
        preds = model.infer(inputs=[x])
        preds = torch.tensor(preds[list(preds.keys())[0]])

        print('preds',preds.shape)
     
    for i in range(batch['nchunk'][0]):
        flat_pred[i*WAV_SIZE:(i+1)*WAV_SIZE] = torch.sigmoid(preds[i]).detach().cpu().numpy()
        
    rem_sz = batch['rem_size'][0]
    if rem_sz > 0:
        flat_pred[-rem_sz:] = torch.sigmoid(preds[-1]).detach().cpu().numpy()[-rem_sz:]

    return flat_pred



In [ ]:
from sklearn.metrics import roc_auc_score,f1_score,precision_score,average_precision_score
        

def test_epoch(model,valDataLoader):

    total_loss=0
    total_step=0
    #model.eval()
    pred_dfs = []

    pbar=tqdm(enumerate(valDataLoader),total=len(valDataLoader))
    for bi,data in pbar :
        pred = validation_step(model,data,bi)
        
        if data['type'][0] == 0:
            preds = []
            for c in range(3):
                preds.append(librosa.resample(pred[:,c].astype(np.float32),orig_sr=100,target_sr=128))
            pred = np.stack(preds,axis=1)
            pred = np.clip(pred,0,1)
            
            pred1 = np.zeros((data['df_length'][0],3))
            pred1[0:data['df_length'][0]] = pred[0:data['df_length'][0]]
            pred=pred1
        
        total_step+=1
        preds_df = pd.DataFrame(pred)
        print('preds_df',preds_df.shape)
        preds_df.columns = TARGET_COLS
        preds_df['Id'] = data['Id'][0]
        preds_df['Id'] = preds_df['Id'] + '_' + preds_df.index.values.astype(str)

        pred_dfs.append(preds_df)
      
    print('len preds_df',len(preds))
    preds = pd.concat(pred_dfs)

    print('preds',preds.shape)
    return preds[TARGET_COLS] , preds[['Id']]

In [ ]:
core = ov.Core()

In [ ]:
GPU=False
def test_loop(params,test_x,ckpt_paths):
    
    #create model
    
    
    models = []
    
    ckpt_paths = glob.glob('/kaggle/input/openvino-model-converter-data/*.onnx')
    ckpt_paths1 = glob.glob('/kaggle/input/gait-wavenet-focal-onnx/*.onnx')
    #ckpt_paths2 = glob.glob('/kaggle/input/wavenet-from-pretrain-v1/*.onnx')
    ckpt_paths2 = glob.glob('/kaggle/input/gait-all-models-v2/wavenet-from-pretrain-v3_wavenet_2000_pretrain*.onnx')
    ckpt_paths.extend(ckpt_paths1)
    ckpt_paths.extend(ckpt_paths2)
    
    print('ckpt_paths',ckpt_paths2)
    
    valDataLoader = getDataLoader(params,test_x)
    
    preds_dfs = []
    id_df = None
    for ckpt_path in ckpt_paths:
        openvino_model = core.read_model(model=ckpt_path)
        compiled_model = core.compile_model(openvino_model, device_name="CPU")
        infer_request = compiled_model.create_infer_request()

        df, id_df = test_epoch(infer_request,valDataLoader)
        print('df',df)
        preds_dfs.append(df)
        
        del infer_request,compiled_model,openvino_model
        gc.collect()
    
    preds = preds_dfs[0].copy()
    for pred_df in preds_dfs:
        for c in TARGET_COLS:
            preds[c] += pred_df[c]
        
    for c in TARGET_COLS:
        preds[c] /= len(ckpt_paths)
    preds['Id'] = id_df
    return preds

In [ ]:
hparams = {
    # Optional hparams
    "backbone": 'wavenet_4096', #'', #'tf_efficientnetv2_b2',
    "learning_rate": [5e-4],
    "max_epochs": 121,
    "batch_size": 8,
    "num_workers": 0,
    "val_sanity_checks": 0,
    "fast_dev_run": False,
    "output_path": f"",
    "gpu": torch.cuda.is_available(),
    'div_factor':10,
    'final_div_factor':20,
}

In [ ]:
import random
seed=42
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = False
    torch.use_deterministic_algorithms = True
    random.seed(0)
    np.random.seed(0)
set_seed(seed)



In [ ]:
import glob
import os
import pandas as pd

tdcsfog_files = glob.glob('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/tdcsfog/*.csv')
tdcsfog_df = pd.DataFrame({'filename':tdcsfog_files, 'type':0} )
defog_files = glob.glob('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/test/defog/*.csv')
defog_df = pd.DataFrame({'filename':defog_files, 'type':1} )

fog_data = pd.concat([tdcsfog_df,defog_df]).reset_index(drop=True)
fog_data['Id'] = fog_data.filename.apply(lambda f:os.path.basename(f).replace('.csv',''))

In [ ]:
fog_data

In [ ]:
%%time
import gc
import glob
version='1'
fn=0



subm = test_loop(hparams,fog_data,None)

In [ ]:
subm

In [ ]:
subm[['Id','StartHesitation','Turn','Walking']].to_csv('./submission.csv',index=False)